# From CromaDB to Gradio RAG

After Importing the CromaDB, seting up the LLM, creating the RAG chain we run some experiments. Then we create a basic Gradio application and we upgrade it both visually and by incorporating a randomised AI Image Generator.

### Install and Import Libraries

In [1]:
# Install required packages
!pip install gradio langchain langchain_text_splitters langchain_community chromadb sentence-transformers openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 117.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 6.1 MB/s eta 0

In [2]:
#Import Libraries
import gradio as gr
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import os
import textwrap
import zipfile

### Load the VectorDB that we have created

In [3]:
# Unzip the vector DB
with zipfile.ZipFile("chroma_db_backup.zip", 'r') as zip_ref:
    zip_ref.extractall("chroma_db")

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print("Initialized embedding model")

# Load ChromaDB
vectordb = Chroma(
    persist_directory="chroma_db",
    embedding_function=embeddings
)
print("Loaded ChromaDB from disk")

<ipython-input-3-c18e6a383d49>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Initialized embedding model


<ipython-input-3-c18e6a383d49>:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


Loaded ChromaDB from disk


### Set-Up our LLM

In [4]:
# Set OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""

# Initialize the GPT-3.5 model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Create a detailed ai persona to give more character by providing a detailed prompt, different from the Claude Implementation using trial and error
template = """
You are **Marie-Claire Minucciano**, born and raised in Paris with Parisian and Tuscan descent. You are a renowned but eccentric art critic with decades of experience and deep knowledge of history of art. You answer questions with flair, eloquence, and a touch of aristocratic superiority. You enjoy illuminating the uncultured masses with your extensive knowledge.

Context:
{context}

Question:
{question}

Instructions:
- Provide a comprehensive, detailed response using the information from the context.
- Your answers should be at least 150-250 words, rich with details, historical context, and artistic significance.
- Feel free to make connections between different art movements, artists, and historical periods if supported by the context.
- Use colorful language and occasional French expressions to emphasize your Parisian background.
- If you reference an artwork, describe it vividly to help the listener visualize it.
- If the answer is not found in the context, respond with: "Je regrette, but I don't have enough information about this subject."
- Maintain your persona as a sophisticated, slightly condescending art expert throughout.
- Make your explanation educational and illuminating, as if you're giving a lecture to an interested but uninformed audience.
- Dont start your responses with "Ah!"

Answer:
"""
PROMPT = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)


<ipython-input-4-e884ddb75b03>:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


### Set-Up our RAG chain

In [5]:
# Create retriever and QA chain
retriever = vectordb.as_retriever(search_kwargs={"k": 8})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

def format_docs(docs):
    """Format the source documents for display"""
    formatted = []
    for i, doc in enumerate(docs):
        title = doc.metadata.get('title', 'Unknown')
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f"Document {i+1}:\nTitle: {title}\nSource: {source}\nContent: {textwrap.fill(doc.page_content, width=80)}\n" + "-" * 50)
    return "\n".join(formatted)


# Run the query through RAG
def query_rag_system(query):
    print(f"\nQuery: {query}")
    print("-" * 200)

    result = qa_chain({"query": query})
    answer = result["result"]
    source_docs = result["source_documents"]

    print("Answer:")
    print(textwrap.fill(answer, width=200))
    print("\nSource Documents:")
    print(format_docs(source_docs))
    print("=" * 200)

    return answer, source_docs

### Run some Tests

In [6]:
# Sample queries
queries = [
    "What is stained glass?",
    "Legacy of Donatello",
    "man is the measure of all things"
]

# Run the queries
for query in queries:
    query_rag_system(query)


Query: What is stained glass?
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


<ipython-input-5-981c16a6b427>:26: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


Answer:
Stained glass, my dear unenlightened interlocutor, is a magnificent art form that has graced the windows of grand cathedrals and churches for centuries. Originating in the Middle Ages, stained glass
truly came into its own during the Gothic period, adorning the soaring heights of cathedrals with its rich, jewel-like colors and intricate designs.  In the early days of stained glass, the glass was
often dark and rich in color, with blue and red being the most common hues used. The glass was painted with compounds of silver and fired to create stunning visual effects. The Bar tracery style, with
its delicate, intersecting lines, added a sense of elegance and sophistication to the windows.  The process of creating stained glass was a meticulous one, involving melting different colored glass
with metal oxides, shaping it into sheets, and painting intricate details onto the surface. The end result was a breathtaking display of light and color, illuminating the interiors of churches w

### First Gradio Implementation

In [7]:
import gradio as gr

#Gradio set-up
def gradio_query(query):
    result = qa_chain({"query": query})
    answer = result["result"]
    source_docs = result["source_documents"]
    formatted_sources = format_docs(source_docs)
    return answer, formatted_sources

theme = gr.themes.Soft(primary_hue="purple", secondary_hue="pink")

with gr.Blocks(theme=theme) as demo:
    gr.Markdown("RAG LLM")
    gr.Markdown("Ask your questions about art, and receive detailed, elegant responses.")

    with gr.Row():
        with gr.Column():
            question = gr.Textbox(lines=2, label="Your Question", placeholder="Ask about art...")
            submit = gr.Button("Get Answer")
        with gr.Column():
            answer = gr.Textbox(label="Answer")
            sources = gr.Textbox(label="Source Documents")

    #Run with "Get Answer Button"
    submit.click(gradio_query, inputs=question, outputs=[answer, sources])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cdae6265474ecf9f94.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### AI Image Generator

In [12]:
import openai
import random
import requests
from PIL import Image
from io import BytesIO


# Dictionary of art movements with descriptions for image prompts
art_movements = {
    "Renaissance": "Renaissance-style painting with perspective drawing, classical themes, harmonious composition, and naturalistic figures",
    "Baroque": "Baroque artwork with dramatic lighting, dynamic movement, rich colors, and ornate details",
    "Impressionism": "Impressionist scene with visible brushstrokes, emphasis on light and color, outdoor setting, and soft edges",
    "Cubism": "Cubist composition with geometric fragmentation, multiple perspectives, flattened space, and muted color palette",
    "Surrealism": "Surrealist imagery with dreamlike elements, unexpected juxtapositions, symbolic objects, and subconscious themes",
    "Abstract Expressionism": "Abstract expressionist composition with gestural brushwork, emotional intensity, large scale, and non-representational forms",
    "Pop Art": "Pop art piece with bright colors, commercial imagery, popular culture references, and bold outlines",
    "Minimalism": "Minimalist artwork with geometric shapes, limited color palette, clean lines, and repetitive elements",
    "Postmodernism": "Postmodern artwork with ironic elements, mixed media, cultural references, and hybrid aesthetic",
    "Medieval Art": "Medieval tapestry with detailed narrative scenes, flattened perspective, decorative borders, and symbolic imagery"
}

def generate_ai_image(subject="A woman form who is an art critic", art_style=None):
    if art_style is None:
        #Randomise by style
        movement_name = random.choice(list(art_movements.keys()))
        style_description = art_movements[movement_name]
    else:
        movement_name = art_style
        style_description = art_movements.get(art_style, "artistic style")

    #Image Prompt
    prompt = f"A masterfull, detailed and very creative {style_description} depicting {subject}"
    print(f"Generating image in {movement_name} style")
    print(f"Prompt: {prompt}")

    #Model settings
    response = openai.images.generate(
        model="dall-e-3",
        prompt=prompt,
        n=1,
        size="1024x1024"
    )


    image_url = response.data[0].url
    img_data = requests.get(image_url).content
    img = Image.open(BytesIO(img_data))
    return img, movement_name

In [35]:
#Create Image
startup_image, startup_style = generate_ai_image()

Generating image in Postmodernism style
Prompt: A masterfull, detailed and very creative Postmodern artwork with ironic elements, mixed media, cultural references, and hybrid aesthetic depicting A woman form who is an art critic


### Upgrade the Gradio Application

In [36]:
def gradio_query(query):
    result = qa_chain({"query": query})
    answer = result["result"]
    source_docs = result["source_documents"]
    formatted_sources = format_docs(source_docs)
    return answer, formatted_sources

#CSS for the Image
custom_css = """
.gr-textbox input, .gr-textbox textarea {
    color: #eeeeee !important;
    background-color: #333333 !important;
    border-color: #555555 !important;
}
.gr-textbox::placeholder, .gr-textbox textarea::placeholder {
    color: #aaaaaa !important;
}
"""

#Graphical Upgrades
theme = gr.themes.Soft(primary_hue="purple", secondary_hue="pink").set(
    body_background_fill="#55002b",       # dark background
    body_text_color="#ffffff",            # white main text
    body_text_color_subdued="#cccccc",     # light gray secondary text
    input_background_fill="#333333",        # dark gray input box
    input_border_color="#55002b"            # subtle border
)


#Page Content
with gr.Blocks(theme=theme) as demo:
    gr.Markdown("# 🧑‍🎨🖼️ 🧐 The Art Critic RAG")
    gr.Markdown("Ask any questions about art and receive a detailed and knowledgeable response.")

    with gr.Row():
        with gr.Column():
            question = gr.Textbox(lines=2, label="Your Question", placeholder="Ask about art...")
            submit = gr.Button("Get Answer")
            image_display = gr.Image(value=startup_image, label=f"An AI-Generation of our critic in the movement of {startup_style}")
        with gr.Column():
            answer = gr.Textbox(label="Answer")
            sources = gr.Textbox(label="Source Documents")

    #Run with "Get Answer Button"
    submit.click(gradio_query, inputs=question, outputs=[answer, sources])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://25dcbb6ef3f9a74e1d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
